In [ ]:
import pandas as pd
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-10-01 11:47:55--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2023-10-01 11:47:55 (9.95 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2023-10-01 11:47:55--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[====

In [ ]:
names = ["class", "message"]

In [ ]:
train_file = pd.read_csv(train_file_path, sep='\t', names=names)
train_file

,class,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [ ]:
test_file = pd.read_csv(test_file_path, sep='\t', names=names)
test_file

,class,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [ ]:
train_message = train_file["message"].values.tolist()
train_label = np.array([0 if x=="ham" else 1 for x in train_file['class'].values.tolist()])
test_message = test_file["message"].values.tolist()
test_label = np.array([0 if x=="ham" else 1 for x in test_file['class'].values.tolist()])

In [ ]:
vocabulary_dict = {}
for messgae in train_message:
  for vocabulary in messgae.split():
    if vocabulary not in vocabulary_dict:
      vocabulary_dict[vocabulary] = 1
    else:
      vocabulary_dict[vocabulary] += 1

In [ ]:
VOCAB_SIZE = len(vocabulary_dict)
MAX_LENGTH = len(max(train_message, key=lambda p: len(p.split())).split())

In [ ]:
encoded_train_message = [one_hot(d, VOCAB_SIZE) for d in train_message]
padded_train_message = pad_sequences(encoded_train_message, maxlen=MAX_LENGTH, padding='post')
encoded_test_message = [one_hot(d, VOCAB_SIZE) for d in test_message]
padded_test_message = pad_sequences(encoded_test_message, maxlen=MAX_LENGTH, padding='post')

In [ ]:
model = Sequential()
embedding_layer = Embedding(VOCAB_SIZE, 100, input_length=MAX_LENGTH)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 171, 100)          1133000   
                                                                 
 flatten_5 (Flatten)         (None, 17100)             0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17101     
                                                                 
Total params: 1150101 (4.39 MB)
Trainable params: 1150101 (4.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
monitor = EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=25, verbose=1, mode='max', restore_best_weights=True)
model.fit(padded_train_message, train_label, validation_data=(padded_test_message, test_label), callbacks=[monitor], epochs=10, verbose=2)

Epoch 1/10
131/131 - 1s - loss: 0.0071 - acc: 0.9986 - val_loss: 0.0431 - val_acc: 0.9885 - 1s/epoch - 8ms/step
Epoch 2/10
131/131 - 1s - loss: 0.0060 - acc: 0.9986 - val_loss: 0.0419 - val_acc: 0.9892 - 1s/epoch - 8ms/step
Epoch 3/10
131/131 - 1s - loss: 0.0049 - acc: 0.9990 - val_loss: 0.0446 - val_acc: 0.9885 - 1s/epoch - 8ms/step
Epoch 4/10
131/131 - 1s - loss: 0.0042 - acc: 0.9990 - val_loss: 0.0466 - val_acc: 0.9878 - 1s/epoch - 8ms/step
Epoch 5/10
131/131 - 1s - loss: 0.0035 - acc: 0.9993 - val_loss: 0.0426 - val_acc: 0.9871 - 1s/epoch - 8ms/step
Epoch 6/10
131/131 - 1s - loss: 0.0032 - acc: 0.9998 - val_loss: 0.0452 - val_acc: 0.9871 - 982ms/epoch - 7ms/step
Epoch 7/10
131/131 - 1s - loss: 0.0028 - acc: 0.9995 - val_loss: 0.0457 - val_acc: 0.9871 - 1s/epoch - 8ms/step
Epoch 8/10
131/131 - 1s - loss: 0.0026 - acc: 0.9998 - val_loss: 0.0461 - val_acc: 0.9878 - 1s/epoch - 8ms/step
Epoch 9/10
131/131 - 1s - loss: 0.0024 - acc: 0.9998 - val_loss: 0.0444 - val_acc: 0.9878 - 1s/epoch 

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  class_dict = {
      0 : "ham",
      1 : "spam",
      }
  encoded_message = [one_hot(pred_text, VOCAB_SIZE)]
  padded_message = pad_sequences(encoded_message, maxlen=MAX_LENGTH, padding='post')
  prediction = [model.predict(padded_message)[0][0], class_dict[np.round(model.predict(padded_message)[0][0])]]
  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 20ms/step
[9.368129e-05, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 20ms/step
[9.368129e-05, 'ham']
1/1 [==============================] - 0s 24ms/step
[0.7164998, 'spam']
1/1 [==============================] - 0s 20ms/step
[6.8129657e-07, 'ham']
1/1 [==============================] - 0s 21ms/step
[0.9999136, 'spam']
1/1 [==============================] - 0s 34ms/step
[0.99995226, 'spam']
1/1 [==============================] - 0s 32ms/step
[0.00023781322, 'ham']
1/1 [==============================] - 0s 32ms/step
[1.5740039e-05, 'ham']
You passed the challenge. Great job!
